Unsure of what to expect. We'll drop, however, the special education and credited & without evaluation features (both 1st and 2nd) to simplify our model. See ceda_i for the reason behind this.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, GaussianNB, ComplementNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier, RandomForestClassifier

import warnings

In [3]:
df = pd.read_csv('../data/train.csv')
df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('.', '')

df.drop(['educational_special_needs', 'curricular_units_1st_sem_(credited)', 'curricular_units_1st_sem_(without_evaluations)', 'curricular_units_2nd_sem_(credited)', 'curricular_units_2nd_sem_(without_evaluations)'],axis = 1, inplace=True)

print(df.shape)
df.head()

(76518, 33)


,id,marital_status,application_mode,application_order,course,daytime/evening_attendance,previous_qualification,previous_qualification_(grade),nacionality,mother's_qualification,...,curricular_units_1st_sem_(approved),curricular_units_1st_sem_(grade),curricular_units_2nd_sem_(enrolled),curricular_units_2nd_sem_(evaluations),curricular_units_2nd_sem_(approved),curricular_units_2nd_sem_(grade),unemployment_rate,inflation_rate,gdp,target
0,0,1,1,1,9238,1,1,126.0,1,1,...,6,14.500000,6,7,6,12.428571,11.1,0.6,2.02,Graduate
1,1,1,17,1,9238,1,1,125.0,1,19,...,4,11.600000,6,9,0,0.000000,11.1,0.6,2.02,Dropout
2,2,1,17,2,9254,1,1,137.0,1,3,...,0,0.000000,6,0,0,0.000000,16.2,0.3,-0.92,Dropout
3,3,1,1,3,9500,1,1,131.0,1,19,...,7,12.591250,8,11,7,12.820000,11.1,0.6,2.02,Enrolled
4,4,1,1,2,9500,1,1,132.0,1,19,...,6,12.933333,7,12,6,12.933333,7.6,2.6,0.32,Graduate


Hmm, without having a formal data science project what to do. I suppose we'll go for default accuracy.

I'll try to reserve comments for something of interest.

In [9]:
X = df.drop(['target', 'id'], axis = 1) #Starting out, at least, with 'everything'. Save id of course as that adds no value.
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.2, #Perhaps subject to change, but we'll go with it for now
                                                    random_state = 26, #Recall that I like this number
                                                    stratify=y) #Lest a dispropportionate amount of one be in any cut.

ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

In [16]:
#Generally I'll only preserve the last bit of parameters I tried in this section of the code, not bothering to record every single iteration...

pipe = Pipeline([
    ('lr', LogisticRegression(max_iter=2500))
])

pipe_params = {'lr__C' : [.85, .95, .9]
              }

gs_lrcv = GridSearchCV(pipe,
                  param_grid=pipe_params,
                  cv=5)

gs_lrcv.fit(X_train, y_train)

print(gs_lrcv.score(X_train, y_train), gs_lrcv.score(X_test, y_test))
print(gs_lrcv.best_score_)
print(gs_lrcv.best_params_)

0.8165452347502206 0.818086774699425
0.815842750975745
{'lr__C': 0.9}


All ready off to a nice start. Ie the simple (albeit I took out a few features already) logistic regression already gave us an overall r2 of .816 on our train split, being slightly underfit in reality. The current leaderboards shows the very best to be at .84179 so it's not like we'd improve all that much in this case by doing something fancier.

Yet, we will anyways!